In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
import pandas as pd

Mounted at /content/drive


In [2]:
os.chdir('/content/drive/My Drive/Zevi')
os.listdir()

['Zevi_search_dataset.json',
 'd2vModel.d2v',
 'new_data.csv',
 'dict_file',
 'lsi_model',
 'tfidf_model',
 'lsi_model.projection',
 'tfidf_model_mm.index',
 'tfidf_model_mm',
 'lsi_model_mm',
 'lsi_model_mm.index',
 'matrix_index',
 'new_data1.json',
 'Zevi model.ipynb',
 'model.png',
 'BERT_model',
 'Classifier Model.ipynb',
 'Final Model.ipynb']

In [3]:
#pip install tf-models-official

In [4]:
#pip install tensorflow_text

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, LSTM, Dropout
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import shutil
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
tf.get_logger().setLevel('ERROR')

In [6]:
data = pd.read_json('new_data1.json')

In [7]:
from spacy.lang.en.stop_words import STOP_WORDS
import spacy
import string
import re

spacy_nlp = spacy.load('en_core_web_sm')

#create list of punctuations and stopwords
punctuations = string.punctuation
stop_words = spacy.lang.en.stop_words.STOP_WORDS

#function for data cleaning and processing
#This can be further enhanced by adding / removing reg-exps as desired.

def spacy_tokenizer(sentence):
 
    #remove distracting single quotes
    sentence = re.sub('\'','',sentence)

    #remove digits adnd words containing digits
    sentence = re.sub('\w*\d\w*','',sentence)

    #replace extra spaces with single space
    sentence = re.sub(' +',' ',sentence)

    #remove unwanted lines starting from special charcters
    sentence = re.sub(r'\n: \'\'.*','',sentence)
    sentence = re.sub(r'\n!.*','',sentence)
    sentence = re.sub(r'^:\'\'.*','',sentence)
    
    #remove non-breaking new line characters
    sentence = re.sub(r'\n',' ',sentence)
    
    #remove punctunations
    sentence = re.sub(r'[^\w\s]',' ',sentence)
    
    #creating token object
    tokens = spacy_nlp(sentence)
    
    #lower, strip and lemmatize
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    
    #remove stopwords, and exclude words less than 2 characters
    tokens = [word for word in tokens if word not in stop_words and word not in punctuations and len(word) > 2]
    
    #return tokens
    return tokens

In [8]:
import numpy as np 
import gensim
import operator
from gensim import corpora
from gensim.similarities import MatrixSimilarity

In [9]:
dictionary = corpora.Dictionary.load('dict_file', mmap=None)

In [10]:
tfidf_model = gensim.models.TfidfModel.load('tfidf_model', mmap=None)
lsi_model = gensim.models.LsiModel.load('lsi_model',mmap=None)

In [11]:
tfidf_corpus = gensim.corpora.MmCorpus('tfidf_model_mm')
lsi_corpus = gensim.corpora.MmCorpus('lsi_model_mm')

In [12]:
index = MatrixSimilarity.load('matrix_index', mmap=None)

In [13]:
idx2cat = {0: 'Computers & Tablets', 1: 'Office Electronics', 2: 'Audio', 3: 'Office & School Supplies', 4: 'MP Exclusives', 5: 'Geek Squad', 6: 'Gift Ideas', 7: 'Cell Phones', 8: 'Magnolia Home Theater', 9: 'Exclusives', 10: 'Car Electronics & GPS', 11: 'Custom Parts', 12: 'Musical Instruments', 13: 'TV & Home Theater', 14: 'Furniture & Decor', 15: 'Housewares', 16: 'Paper', 17: 'Appliances', 18: 'Toys, Games & Drones', 19: 'Star Wars', 20: 'Beverage & Wine Coolers', 21: 'Cameras & Camcorders', 22: 'Household Essentials', 23: 'Video Games', 24: 'Entertainment Gift Cards', 25: 'Avengers: Age of Ultron', 26: 'Office Furniture & Storage', 27: 'MP Pending', 28: 'Name Brands', 29: 'DC Comics', 30: 'Health, Fitness & Beauty', 31: 'Carfi Instore Only', 32: 'Scanners, Faxes & Copiers', 33: 'Telephones & Communication', 34: 'Movies & Music', 35: 'Smart Home', 36: 'Wearable Technology', 37: 'Batteries & Power', 38: 'In-Store Only'}

In [21]:
classifier_model = tf.keras.models.load_model('BERT_model',compile=False)

In [90]:
x_test = np.array(['nokia'])
ans = classifier_model.predict(x_test)
max_index = np.argmax(ans)
print(idx2cat[max_index])

Cell Phones


In [79]:
from operator import itemgetter

def search_similar_products(search_term):

    query_bow = dictionary.doc2bow(spacy_tokenizer(search_term))
    query_tfidf = tfidf_model[query_bow]
    query_lsi = lsi_model[query_tfidf]

    index.num_best = 5

    product_list = index[query_lsi]

    category = idx2cat[np.argmax(classifier_model.predict(np.array([search_term])))]

    probability = np.max(classifier_model.predict(np.array([search_term])))

    for i in range(0,len(product_list)):
      if(len(data['categories'][product_list[i][0]])!=0 and data['categories'][product_list[i][0]][0]==category):
        product_list[i] = (product_list[i][0],product_list[i][1]+2*probability/10)


    product_list.sort(key=itemgetter(1), reverse=True)
    names = []

    for j, product in enumerate(product_list):

        names.append (
            {
                'Relevance': round((product[1] * 100),2),
                'Name': data['name'][product[0]], 
            }

        )
        if j == (index.num_best-1):
            break

    return pd.DataFrame(names, columns=['Relevance','Name'])

In [100]:
search_similar_products('macbook')

,Relevance,Name
0,96.31,"Samsonite - Aramon NXT Shuttle Case for 13"" Ap..."
1,95.75,"Incase - Sling Sleeve for 13"" Apple® MacBook® ..."
2,92.83,"Speck - SmartShell Upper Shield Case for 13.3""..."
3,87.26,"Speck - SeeThru Case for 13"" Apple® MacBook® P..."
4,86.81,"Speck - SmartShell Glitter Case for 13"" Apple®..."
